# Migrate Functions to a New Catalog
This notebook reads all user-defined functions from a source catalog and generates SQL commands to recreate them in a destination catalog. Specify the catalog names below and run all cells.

In [ ]:

# Configure source and destination catalogs
try:
    dbutils.widgets.text("1.source_catalog", "source_catalog")
    dbutils.widgets.text("2.destination_catalog", "destination_catalog")
    source_catalog = dbutils.widgets.get("1.source_catalog")
    destination_catalog = dbutils.widgets.get("2.destination_catalog")
except NameError:
    # When running as a standard Python script (e.g., for testing), define values here
    source_catalog = "source_catalog"
    destination_catalog = "destination_catalog"


## Read Functions From the Source Catalog

In [ ]:

functions = []
schemas_df = spark.sql(f"SHOW SCHEMAS IN `{source_catalog}`")
for row in schemas_df.collect():
    schema = row[0]
    funcs_df = spark.sql(f"SHOW USER FUNCTIONS IN `{source_catalog}`.`{schema}`")
    for f in funcs_df.collect():
        func_name = f[0]
        spark.sql(f"USE CATALOG `{source_catalog}`")
        spark.sql(f"USE SCHEMA `{schema}`")
        df = spark.sql(f"DESCRIBE FUNCTION EXTENDED `{func_name}`")
        body = [r.function_desc for r in df.collect() if r.function_desc.startswith('Body:')][0][6:]
        dest_stmt = f"CREATE FUNCTION `{destination_catalog}`.`{schema}`.`{func_name}` {body}"
        functions.append(dest_stmt)


## Display CREATE FUNCTION Commands

In [ ]:

for cmd in functions:
    print(cmd)
